In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import OrdinalEncoder
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings("ignore")

# Peer-graded Assignment: Построение baseline-решений

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:

* Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
* Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
* Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)

In [2]:
# Загрузим данные  
data = pd.read_csv('../Data/train_dataset.csv')
data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
0,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,...,z9ub4Lm,LM8l689qOp,NaN,NaN,5Acm,vJ_w8kB,WfJ2BB2SFSqauljlfOB,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,2401.0,21.0,NaN,NaN,NaN,...,v5hz20V,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,343.0,0.0,NaN,NaN,NaN,...,20HE4Qn,LM8l689qOp,NaN,NaN,Xa3G,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,826.0,7.0,NaN,NaN,NaN,...,4XQyovK,LM8l689qOp,NaN,ELof,453m,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
4,NaN,NaN,NaN,NaN,3960.0,NaN,NaN,NaN,NaN,1641096.0,...,LTMqFbB,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


# Обработка данных 

In [4]:
# Удалим все столбцы, состоящие только из NaN
deleted = []
for var in data.columns:
    if data[var].isna().sum() == data.shape[0]:
        data.drop(var, axis=1,inplace=True)
        deleted.append(var)
# В категориальных признаках заменим пропуски на новую категорию
data.loc[:,'Var191':] = data.loc[:,'Var191':].fillna('missing_value')
np.array(deleted)

array(['Var8', 'Var15', 'Var20', 'Var31', 'Var32', 'Var39', 'Var42',
       'Var48', 'Var52', 'Var55', 'Var79', 'Var141', 'Var167', 'Var169',
       'Var175', 'Var185', 'Var209', 'Var230'], dtype='<U6')

In [6]:
# Проверим есть ли значение 0.0001 в данных, чтобы в дальнейшем заполнить им пропуски 
0.0001 in data.loc[:, 'Var1':'Var190'].values

False

In [7]:
data.loc[:, 'Var1':'Var190'] = data.loc[:, 'Var1':'Var190'].fillna(0.0001)

In [10]:
# Выведем соотношение классов
not_churn = data.label.value_counts()[-1]
churn = data.label.value_counts()[1]

print('Доля класса "отток": ', churn/data.shape[0])
print('Доля класса "не отток": ', not_churn/data.shape[0])

Доля класса "отток":  0.073
Доля класса "не отток":  0.927


In [28]:
# Создадим датасеты
cat_features = data.loc[:,'Var191':'Var229']
numeric_features = data.loc[:, 'Var1':'Var190']
y = data['label']

In [30]:
# Закодируем категориальные переменные
OE_enc = OrdinalEncoder()
OE_enc.fit(cat_features)
oe_features = pd.DataFrame(OE_enc.transform(cat_features), columns = cat_features.columns)
X_oe = pd.concat([numeric_features,oe_features], axis=1)
X_oe.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0001,0.0001,0.0001,0.0001,0.0001,819.0000,7.0000,0.0001,1.000000e-04,0.0001,...,311.0,2.0,3344.0,0.0,1.0,2.0,2.0,6.0,16.0,1.0
1,0.0001,0.0001,0.0001,0.0001,0.0001,2401.0000,21.0000,0.0001,1.000000e-04,0.0001,...,2075.0,4.0,3170.0,0.0,1.0,1.0,7.0,2.0,2.0,2.0
2,0.0001,0.0001,0.0001,0.0001,0.0001,343.0000,0.0000,0.0001,1.000000e-04,0.0001,...,2618.0,4.0,86.0,0.0,1.0,2.0,13.0,2.0,8.0,1.0
3,0.0001,0.0001,0.0001,0.0001,0.0001,826.0000,7.0000,0.0001,1.000000e-04,0.0001,...,2269.0,4.0,208.0,0.0,1.0,0.0,1.0,2.0,8.0,1.0
4,0.0001,0.0001,0.0001,0.0001,3960.0000,0.0001,0.0001,0.0001,1.641096e+06,0.0001,...,1002.0,4.0,1156.0,0.0,1.0,2.0,7.0,2.0,8.0,1.0


# baseline модели

In [29]:
# Зададим параметры моделей
# Основаная метрика - precision, отсальные метрики вспомогательные
scorings = ['precision', 'recall','f1','roc_auc']
# стратегия кросс-валиадции - stratified K-Fold с 5 фолдами была выбрана ранее
cv = StratifiedKFold(n_splits=5)

## Логистическая регрессия

In [40]:
lr_model = LogisticRegression()
scores_lr = cross_validate(lr_model, X_oe, y, cv=cv, scoring=scorings)

In [41]:
print('Средние значения метрик\n')
for key in scores_lr.keys():
    print(key, ':',scores_lr[key].mean())

Средние значения метрик

fit_time : 24.321565914154053
score_time : 0.059439659118652344
test_precision : 0.24543859649122807
test_recall : 0.005479452054794521
test_f1 : 0.01059247820776546
test_roc_auc : 0.5288698260702517


## Случайный лес

In [38]:
rfc_model = RandomForestClassifier()
scores_rfc = cross_validate(rfc_model, X_oe, y, cv=cv, scoring=scorings)

In [39]:
print('Средние значения метрик\n')
for key in scores_rfc.keys():
    print(key, ':',scores_rfc[key].mean())

Средние значения метрик

fit_time : 1.1875423908233642
score_time : 0.1405796527862549
test_precision : 0.3371428571428571
test_recall : 0.00502283105022831
test_f1 : 0.009889765178328984
test_roc_auc : 0.6044357175813042


## Градиентный бустинг

In [44]:
gb_model = GradientBoostingClassifier()
scores_gb = cross_validate(gb_model, X_oe, y, cv=cv, scoring=scorings)

In [45]:
print('Средние значения метрик\n')
for key in scores_gb.keys():
    print(key, ':',scores_gb[key].mean())

Средние значения метрик

fit_time : 17.514989614486694
score_time : 0.1750720977783203
test_precision : 0.5023529411764706
test_recall : 0.01598173515981735
test_f1 : 0.030892211904491727
test_roc_auc : 0.7312983651293267


Лучший результат получается при использовании градиентного бустнинга, так как данный классификатор с параметрами по умолчанию дает наилучший результат как по целевой метрике, так и по вспомогательным.

При этом величина данных метрик показывает, что модели пока работают довольно плохо. Их нужно улучшать как путем работы с данными, так и подбором параметров модели, что будет сделано в дальнейшем.